In [ ]:
import json
import sys
from pathlib import Path
import re

from tqdm import tqdm_notebook
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [ ]:
firefox_profile = webdriver.FirefoxProfile()
# firefox_profile.set_preference('permissions.default.image', 2)  # отключить загрузку картинок
firefox_profile.set_preference('dom.ipc.plugins.enabled.libflashplayer.so', 'false')

driver = webdriver.Firefox(executable_path='/home/anastasia/Downloads/geckodriver', firefox_profile=firefox_profile)

In [ ]:
url = 'http://ege.fipi.ru/os11/xmodules/qprint/openlogin.php?proj=AF0ED3F2557F8FFC4C06F80B6803FD26'
driver.get(url)

In [ ]:
def parse_page(raw_html):
    soup = BeautifulSoup(raw_html, 'lxml')
    try:
        t = soup.find_all('td', {'class':'cell_1'})
        for el in t:
            forms = el.find_all("form")
            tasks = []
            for form in forms:
                blocks = form.select('form > table > tbody > tr')
                d = []
                for block in blocks:
                    p = block.find_all("p")
                    p = [x.text.strip() for x in p if x]
                    p = [re.sub('(<(m|/m).*?>)+', '', x).replace('\xa0', '').replace('\xad', '')for x in p if x]
                    if p:
                        d.append(p)
                tasks.append(d)
            d1 = {str(soup.select_one('td.header').text):tasks}
            with open('rus_tasks_ege.jsonl', 'a', encoding='utf8') as fout:
                print(json.dumps(d1, ensure_ascii=False), file=fout, flush=True)
    except AttributeError:
        print('error')

In [ ]:
headers_len = len(driver.find_elements_by_css_selector('td.coursebody a'))
for i in range(headers_len):
    driver.find_elements_by_css_selector('td.coursebody a')[i].click()
    parse_page(driver.page_source)
    len_pages = int(len(driver.find_elements_by_css_selector('span.Walk a')) / 2)
    for k in range(len_pages):
        driver.find_elements_by_css_selector('span.Walk a')[k].click()
        parse_page(driver.page_source)